In [1]:
import wrds
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
db = wrds.Connection()
db.create_pgpass_file()

Loading library list...
Done
Enter your WRDS username [jonkq]:jonkq
Enter your password:········


In [3]:
stocks = db.raw_sql("""
                select a.permno, a.date, a.ret, b.exchcd, b.shrcd, a.shrout, a.prc 

                from crsp.msf as a left join crsp.msenames as b 
                
                on a.permno = b.permno and b.namedt <= a.date and a.date<=b.nameendt
                
                where a.date between '1980-01-01' and '2019-12-31'
                and b.exchcd between 1 and 2
                and b.shrcd between 10 and 11""",date_cols=['date'])

stocks.to_csv('stocks.csv')

In [75]:
# change table format from long to wide
stocks_ret = pd.pivot_table(data=stocks, values='ret', index='date', columns='permno')
stocks_shrout = pd.pivot_table(data=stocks, values='shrout', index='date', columns='permno')
stocks_prc = pd.pivot_table(data=stocks, values='prc', index='date', columns='permno')

In [76]:
s = stocks_ret.apply(sum)
s = list(s.isna())
s_i = []
for i in range(len(s)):
    if not s[i]:
        s_i.append(i)

In [77]:
# removing stocks containing NaN values
stocks_ret = stocks_ret.iloc[:,s_i]
stocks_shrout = stocks_shrout.iloc[:,s_i]
stocks_prc = stocks_prc.iloc[:,s_i]

In [78]:
stocks_prc = stocks_prc.abs()

In [88]:
for j in range(stocks_shrout.shape[1]):
    for i in range(stocks_shrout.shape[0]):
        if pd.isna(stocks_shrout.iloc[i,j]) or pd.isnull(stocks_shrout.iloc[i,j]):
            if i == 0:
                s = 1
                while pd.isna(stocks_shrout.iloc[i+s,j]) or pd.isnull(stocks_shrout.iloc[i+s,j]):
                    s += 1
                stocks_shrout.iloc[i:(i+s+1),j] = stocks_shrout.iloc[i+s,j]
            else:
                stocks_shrout.iloc[i,j] = stocks_shrout.iloc[i-1,j]

In [90]:
for j in range(stocks_prc.shape[1]):
    for i in range(stocks_prc.shape[0]):
        if pd.isna(stocks_prc.iloc[i,j]) or pd.isnull(stocks_prc.iloc[i,j]):
            if i == 0:
                s = 1
                while pd.isna(stocks_prc.iloc[i+s,j]) or pd.isnull(stocks_prc.iloc[i+s,j]):
                    s += 1
                stocks_prc.iloc[i:(i+s+1),j] = stocks_prc.iloc[i+s,j]
            else:
                stocks_prc.iloc[i,j] = stocks_prc.iloc[i-1,j]

In [92]:
stocks_mcap = pd.DataFrame(stocks_shrout.values*stocks_prc.values, columns=stocks_shrout.columns, index=stocks_shrout.index)

In [102]:
last_mcap = stocks_mcap.iloc[len(stocks_mcap)-1,:]
permnos = np.array(last_mcap.index)
mcap_ord = np.array(np.argsort(last_mcap))
mcap_sort = np.sort(last_mcap)

In [105]:
len(mcap_ord)

271

In [103]:
mcap_groups = 

array([9.60750000e+02, 1.04400000e+03, 2.23062500e+03, 2.46684375e+03,
       2.63789011e+03, 3.09956250e+03, 3.21637500e+03, 4.31775000e+03,
       4.38200000e+03, 5.31562500e+03, 5.91250000e+03, 5.99137500e+03,
       6.24510011e+03, 6.77250000e+03, 7.06400000e+03, 7.49375000e+03,
       8.30250000e+03, 9.78600000e+03, 9.92816981e+03, 1.13557500e+04,
       1.17633750e+04, 1.26652812e+04, 1.37523750e+04, 1.51510399e+04,
       1.52638750e+04, 1.69830000e+04, 1.70002592e+04, 1.73755000e+04,
       1.74287500e+04, 1.75964744e+04, 1.78733750e+04, 1.88515196e+04,
       2.04802500e+04, 2.12467500e+04, 2.19676250e+04, 2.21556606e+04,
       2.22430000e+04, 2.24191084e+04, 2.60730215e+04, 2.69550000e+04,
       2.83353750e+04, 2.90257500e+04, 2.97620312e+04, 3.14243999e+04,
       3.19435804e+04, 3.26340000e+04, 3.52217986e+04, 3.59707500e+04,
       3.61568750e+04, 3.62256250e+04, 3.92740000e+04, 4.00365000e+04,
       4.05899375e+04, 4.09850000e+04, 4.14686780e+04, 4.18470000e+04,
      